### Select patients for CFR model: Combine file names, cfr measurements and views ###

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

from bokeh.io import output_notebook, reset_output, show, output_file, save
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot
from bokeh.models import ColumnDataSource, HoverTool, Legend

from bokeh.palettes import Category10

In [2]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
cfr_meta_date = '200219'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+cfr_meta_date)
print(meta_dir)

/mnt/obi0/andreas/data/cfr/metadata_200219


In [3]:
# CFR measurements matched with echo studies
study_cfr_file = os.path.join(meta_dir, 'BWH_2020-Feb-4_pet_match365_diff_'+cfr_meta_date+'.parquet')
study_cfr_df = pd.read_parquet(study_cfr_file)
print('Number of matched CFR-ECHO studies: {}'.format(len(study_cfr_df.study.unique())))
study_cfr_df.head()

Number of matched CFR-ECHO studies: 2751


,study,mrn,echo_study_date,mrnstudydateid,days_post_cfr,reportID,subjectID,cfr_study_date,Reason_post_MI_eval,Reason_post_CABG,Reason_post_PCI,Reason_myocardial_viability,reason_postHeartTransplant,hx_recent_MI_yes,troponin,hx_prior_MI_yes,hx_prior_transplant,hx_prior_transplant_yes,hx_CABG,hx_CABG_yes,hx_PCI,hx_PCI_yes,hx_CHF,hx_CHF_yes,hx_cardiomyopathy_yes,...,perf_seg_8,perf_seg_9,perf_seg_10,perf_seg_11,perf_seg_12,perf_seg_13,perf_seg_14,perf_seg_15,perf_seg_16,perf_seg_17,model_rest_mbf,num_unaffected_segs,global_CFR,model_cfr,Stress,"Regional defect(normal, probably normal, abnormal)",CABG/Transplant,Comment,calc_res,Regional defect(yes/no),Unnamed: 7,Unnamed: 8,Unnamed: 9,rahul_echo_date,difference(days)
0,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.936460,16.0,1.716143,1.707419,NaN,None,None,None,NaN,None,None,None,None,2018-04-24,1
1,48b091b44637c987_4903a58c1bcb23767fc2818c6e63,35014216,2019-04-25,2433,352,162416.0,7843.0,2018-05-08,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,NaT,False,NaT,False,False,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.617024,7.0,2.644905,2.996385,NaN,None,None,None,NaN,None,None,None,None,2019-04-25,352
2,48b091b714b41ffe_4903a582ec746dce5e9b6c1f4070,35029545,2017-05-16,2434,-69,156019.0,7376.0,2017-07-24,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,NaT,False,NaT,False,False,...,0.0,3.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.574958,4.0,2.698341,2.631967,NaN,None,None,None,NaN,None,None,None,None,2017-07-21,3
3,48b09412e8754782_4903a582ec78ec0366e9db9e17cd,35583509,2017-09-06,2437,-336,164488.0,7590.0,2018-08-08,False,False,False,False,False,False,0.0,True,NaT,False,NaT,False,2015-07-12,True,NaT,False,False,...,2.0,2.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,6.0,0.544319,3.0,1.994660,2.063722,NaN,None,None,None,NaN,None,None,None,None,2017-09-06,336
4,48b095533cbfdf01_4903a582ec760da4fd52ba61f21c,35401116,2017-07-07,2436,-643,NaN,NaN,2019-04-11,None,None,None,None,None,None,NaN,None,NaT,None,NaT,None,NaT,None,NaT,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.790000,NaN,NaN,1.110000,0.88,None,None,None,NaN,normal,None,1.113924051,0.003924051,2019-04-09,2


In [5]:
# Meta data for all echo videos (or just this subset)
echo_meta_file = os.path.join(meta_dir, 'echo_BWH_meta_cfr_'+cfr_meta_date+'.parquet')
echo_meta_df = pd.read_parquet(echo_meta_file)
echo_meta_df = echo_meta_df.astype({'mrn': 'int64'})
print('Number of studies in meta data {}'.format(len(echo_meta_df.study.unique())))
echo_meta_df.head()

Number of studies in meta data 2750


,filename,dir,study,mrn,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal
0,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09016b...,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02 09:49:36,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,BWH,Vivid E95,GE Vingmed Ultrasound,0.0,33.500000,71.0,78.0,0.013469,0.013469,1.115408e-13,3.261696e-13,1.222044e-10,2.565391e-08,7.528357e-13,4.774701e-10,2.708740e-12,3.670912e-09,1.567517e-11,1.610227e-10,1.509277e-07,2.566710e-06,1.363240e-13,2.349999e-08,7.267388e-09,2.949915e-12,9.663496e-15,1.294855e-04,9.732323e-06,1.612119e-12,9.998580e-01,4.653681e-10,5.331582e-11,3.434301e-11,4.860822e-09,5.479821e-14,2.986510e-08
1,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09016b...,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02 09:49:36,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,BWH,Vivid E95,GE Vingmed Ultrasound,0.0,33.371233,72.0,77.0,0.027226,0.027226,7.617776e-10,2.469505e-10,8.256379e-10,1.450411e-07,1.366147e-09,3.045583e-11,2.590847e-12,3.514123e-09,1.119247e-12,1.174291e-11,1.653268e-09,6.647998e-09,7.400897e-07,4.554047e-08,2.202479e-12,1.033245e-11,1.524424e-10,1.032910e-10,1.392449e-09,8.828201e-10,5.749271e-07,9.999073e-01,7.699048e-07,6.217265e-08,8.135313e-09,1.209222e-09,9.039690e-05
2,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09016b...,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02 09:49:36,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,BWH,Vivid E95,GE Vingmed Ultrasound,0.0,33.533333,76.0,77.0,0.027251,0.027251,1.779165e-08,5.061842e-11,1.852694e-09,2.660267e-07,3.196280e-10,2.934359e-09,3.283927e-07,3.254719e-09,1.172012e-07,4.847203e-10,3.966162e-09,1.291069e-09,1.924162e-08,6.683372e-07,9.999976e-01,1.350526e-09,4.165070e-10,1.177100e-10,1.866018e-09,3.144208e-09,3.432988e-10,9.421681e-10,8.573890e-07,4.335133e-08,1.434178e-10,4.363453e-12,2.205031e-09
3,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09016b...,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02 09:49:36,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,BWH,Vivid E95,GE Vingmed Ultrasound,0.0,33.708920,72.0,78.0,0.028956,0.028956,1.000000e+00,2.050265e-13,4.028317e-15,1.941936e-14,2.563091e-18,2.315560e-14,1.126485e-08,5.550321e-15,1.119701e-19,4.695120e-16,9.429649e-14,1.808776e-17,1.098487e-13,1.432877e-15,2.962554e-15,2.005398e-15,3.412526e-12,1.698335e-14,8.875975e-14,8.122525e-18,1.691228e-16,5.149206e-17,7.832482e-14,1.353168e-15,1.369931e-19,3.447312e-22,2.979074e-17
4,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,/mnt/obi0/phi/echo/npyFiles/BWH/48b0/48b09016b...,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02 09:49:36,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f_...,BWH,Vivid E95,GE Vingmed Ultrasound,0.0,33.403756,72.0,78.0,0.023838,0.023838,1.130372e-15,3.884820e-06,1.672595e-12,4.836035e-07,9.999956e-01,4.835767e-11,1.211003e-13,1.254464e-09,3.868239e-12,6.760433e-14,1.656497e-10,1.586072e-09,2.450811e-11,3.360560e-12,2.185993e-14,4.238282e-11,6.147999e-12,9.961027e-12,3.035175e-11,3.153882e-10,3.031236e-14,4.888586e-13,3.554045e-12,1.007157e-11,7.237515e-15,2.361346e-18,9.150120e-12


In [6]:
print('Total number of patients in meta data {}'.format(len(echo_meta_df.mrn.unique())))
print('Total number of studies {}'.format(len(echo_meta_df.study.unique())))
print('Total number of files in meta data {}'.format(len(echo_meta_df.filename.unique())))

temp = echo_meta_df.loc[echo_meta_df.a4c.isnull()].reset_index(drop = True)
print()
print('Studies without view classification {}'.format(len(temp.study.unique())))

# Remove meta rows without view classification
echo_meta_all_views = echo_meta_df.loc[~echo_meta_df.a4c.isnull()]
# Remove rows without frame_time
echo_meta_all_views = echo_meta_all_views.loc[~echo_meta_all_views.frame_time.isnull()].reset_index(drop = True)

print()
print('After removal of rows without view clasification:')
print('Total number of patients in meta data {}'.format(len(echo_meta_all_views.mrn.unique())))
print('Total number of studies {}'.format(len(echo_meta_all_views.study.unique())))
print('Total number of files in meta data {}'.format(len(echo_meta_all_views.filename.unique())))

Total number of patients in meta data 1626
Total number of studies 2750
Total number of files in meta data 126420

Studies without view classification 5

After removal of rows without view clasification:
Total number of patients in meta data 1626
Total number of studies 2750
Total number of files in meta data 126144


In [7]:
view_dict = {'view_a2c': ['a2c', 'a2c_laocc', 'a2c_lvocc_s'],
             'view_a3c': ['a3c', 'a3c_laocc', 'a3c_lvocc_s'],
             'view_a4c': ['a4c', 'a4c_far', 'a4c_laocc', 'a4c_lvocc_s', 'a4c_rv', 'a4c_rv_laocc'],
             'view_plax': ['plax_far', 'plax_lac', 'plax_laz', 'plax_laz_ao', 'plax_plax'],
             'view_psax': ['psax_avz', 'psax_az', 'psax_mv', 'psax_pap'],
             'view_other': ['other', 'a5c', 'apex', 'rvinf', 'subcostal', 'suprasternal']}

In [8]:
# Combine some of the view columns
echo_meta_sum_views = echo_meta_all_views.copy()
for view in view_dict.keys():
    cols = view_dict[view]
    echo_meta_sum_views[view] = echo_meta_sum_views[cols].sum(axis =1)
    # Drop the old columns
    echo_meta_sum_views = echo_meta_sum_views.drop(columns = cols)

# Sum up the new view columns as a consistency check
#echo_meta_sum_views = echo_meta_sum_views.assign(sum_views = echo_meta_sum_views[list(view_dict.keys())].\                                                 sum(axis = 1))

# Get the maxiumum view classification score for each row
#echo_meta_sum_views = echo_meta_sum_views.assign(max_view = echo_meta_sum_views[list(view_dict.keys())].\
#                                                 idxmax(axis = 1))

# Get the maxiumum view classification score for each row
view_list = [item for sublist in view_dict.values() for item in sublist]
echo_meta_sum_views = echo_meta_all_views.assign(max_view = echo_meta_all_views[view_list].\
                                                 idxmax(axis = 1))

In [9]:
# Left join in the PET CFR table, thereby filtering those studies that have CFR values
files_cfr = study_cfr_df.merge(right = echo_meta_sum_views, on = ['mrn', 'study'], how = 'left')
files_cfr.tail(2)

,study,mrn,echo_study_date,mrnstudydateid,days_post_cfr,reportID,subjectID,cfr_study_date,Reason_post_MI_eval,Reason_post_CABG,Reason_post_PCI,Reason_myocardial_viability,reason_postHeartTransplant,hx_recent_MI_yes,troponin,hx_prior_MI_yes,hx_prior_transplant,hx_prior_transplant_yes,hx_CABG,hx_CABG_yes,hx_PCI,hx_PCI_yes,hx_CHF,hx_CHF_yes,hx_cardiomyopathy_yes,...,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view
126143,4f264df7bb568062_4903a58c1a395137157141a56e7d,40611006,2019-10-05,2469,-12,NaN,NaN,2019-10-17,None,None,None,None,None,None,NaN,None,NaT,None,NaT,None,NaT,None,NaT,None,None,...,1.045394e-03,1.002030e-08,6.245052e-05,3.078212e-07,1.191865e-04,1.786875e-07,3.655481e-05,4.006131e-08,1.988119e-06,1.141177e-04,8.287738e-03,2.204315e-04,1.786716e-02,3.257524e-03,4.848562e-09,5.523269e-08,0.012004,9.572906e-03,0.003405,8.297683e-04,1.022397e-04,9.365516e-01,1.984576e-07,6.349634e-03,rvinf
126144,4f264df7bb568062_4903a58c1a395137157141a56e7d,40611006,2019-10-05,2469,-12,NaN,NaN,2019-10-17,None,None,None,None,None,None,NaN,None,NaT,None,NaT,None,NaT,None,NaT,None,None,...,1.069302e-07,1.200457e-11,1.132369e-09,2.720129e-13,2.567232e-11,5.291282e-11,2.164626e-09,1.950443e-13,2.352980e-09,2.009301e-09,1.414583e-07,3.743202e-08,1.011342e-10,2.467321e-07,6.970213e-11,3.645742e-10,0.999995,6.899067e-11,0.000004,7.671266e-09,1.233400e-12,1.433359e-11,7.481736e-09,2.660287e-09,plax_plax


In [11]:
# Let's see what we have
df_stat = pd.DataFrame()
for view in view_list:
    df = files_cfr[files_cfr.max_view == view]
    
    df_dict = {'view': [view],
               'patients': [len(df.mrn.unique())],
               'echo studies': [len(df.study.unique())],
               'unique CFR values': [len(df.model_cfr.unique())]}    
    df_stat = pd.concat([df_stat, pd.DataFrame(df_dict)], ignore_index = True)

df_stat = df_stat.sort_values(by = 'view')
print(df_stat.head(100))

            view  patients  echo studies  unique CFR values
0            a2c      1337          2040               1363
1      a2c_laocc      1007          1423               1017
2    a2c_lvocc_s         8             9                  8
3            a3c      1249          1877               1274
4      a3c_laocc       756           976                759
5    a3c_lvocc_s        34            35                 34
6            a4c      1481          2344               1512
7        a4c_far       828          1122                834
8      a4c_laocc      1207          1739               1219
9    a4c_lvocc_s        77            78                 77
10        a4c_rv       960          1344                970
11  a4c_rv_laocc        38            40                 38
22           a5c      1139          1625               1160
23          apex       769          1076                775
21         other      1614          2715               1652
12      plax_far      1199          1730

In [14]:
match_view_filename = 'BWH_2020-Feb-19_pet_match365_diff_files_'+cfr_meta_date+'.parquet'
files_cfr.to_parquet(os.path.join(meta_dir, match_view_filename))

# Let's do a .csv file as well
match_view_filename_csv = 'BWH_2020-Feb-19_pet_match365_diff_files_'+cfr_meta_date+'.csv'
files_cfr.to_csv(os.path.join(meta_dir, match_view_filename_csv), index = False)

In [15]:
print(files_cfr.shape)
files_cfr.head()

(126145, 401)


,study,mrn,echo_study_date,mrnstudydateid,days_post_cfr,reportID,subjectID,cfr_study_date,Reason_post_MI_eval,Reason_post_CABG,Reason_post_PCI,Reason_myocardial_viability,reason_postHeartTransplant,hx_recent_MI_yes,troponin,hx_prior_MI_yes,hx_prior_transplant,hx_prior_transplant_yes,hx_CABG,hx_CABG_yes,hx_PCI,hx_PCI_yes,hx_CHF,hx_CHF_yes,hx_cardiomyopathy_yes,...,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,max_view
0,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,2.565391e-08,7.528357e-13,4.774701e-10,2.708740e-12,3.670912e-09,1.567517e-11,1.610227e-10,1.509277e-07,2.566710e-06,1.363240e-13,2.349999e-08,7.267388e-09,2.949915e-12,9.663496e-15,1.294855e-04,9.732323e-06,1.612119e-12,9.998580e-01,4.653681e-10,5.331582e-11,3.434301e-11,4.860822e-09,5.479821e-14,2.986510e-08,psax_avz
1,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,1.450411e-07,1.366147e-09,3.045583e-11,2.590847e-12,3.514123e-09,1.119247e-12,1.174291e-11,1.653268e-09,6.647998e-09,7.400897e-07,4.554047e-08,2.202479e-12,1.033245e-11,1.524424e-10,1.032910e-10,1.392449e-09,8.828201e-10,5.749271e-07,9.999073e-01,7.699048e-07,6.217265e-08,8.135313e-09,1.209222e-09,9.039690e-05,psax_az
2,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,2.660267e-07,3.196280e-10,2.934359e-09,3.283927e-07,3.254719e-09,1.172012e-07,4.847203e-10,3.966162e-09,1.291069e-09,1.924162e-08,6.683372e-07,9.999976e-01,1.350526e-09,4.165070e-10,1.177100e-10,1.866018e-09,3.144208e-09,3.432988e-10,9.421681e-10,8.573890e-07,4.335133e-08,1.434178e-10,4.363453e-12,2.205031e-09,other
3,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,1.941936e-14,2.563091e-18,2.315560e-14,1.126485e-08,5.550321e-15,1.119701e-19,4.695120e-16,9.429649e-14,1.808776e-17,1.098487e-13,1.432877e-15,2.962554e-15,2.005398e-15,3.412526e-12,1.698335e-14,8.875975e-14,8.122525e-18,1.691228e-16,5.149206e-17,7.832482e-14,1.353168e-15,1.369931e-19,3.447312e-22,2.979074e-17,a2c
4,48b09016b25a7f36_4903a58dd44b7b81d524809aba5f,35133560,2018-11-02,2435,191,162094.0,7817.0,2018-04-25,False,False,False,False,False,False,0.0,False,NaT,False,NaT,False,2015-01-01,True,NaT,False,False,...,4.836035e-07,9.999956e-01,4.835767e-11,1.211003e-13,1.254464e-09,3.868239e-12,6.760433e-14,1.656497e-10,1.586072e-09,2.450811e-11,3.360560e-12,2.185993e-14,4.238282e-11,6.147999e-12,9.961027e-12,3.035175e-11,3.153882e-10,3.031236e-14,4.888586e-13,3.554045e-12,1.007157e-11,7.237515e-15,2.361346e-18,9.150120e-12,a3c_laocc


In [16]:
view_list_plot = [v for v in view_list if 'a4c' in v]
# For the paper: Make a graph with the days of echo before or after CFR
def style(p):
    # Title 
    p.title.align = 'center'
    p.title.text_font_size = '11pt'
    #p.title.text_font = 'serif'

    # Axis titles
    p.xaxis.axis_label_text_font_size = '11pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '11pt'
    p.yaxis.axis_label_text_font_style = 'bold'

    # Tick labels
    p.xaxis.major_label_text_font_size = '11pt'
    p.yaxis.major_label_text_font_size = '11pt'
    
    return p

def make_dataset(df = files_cfr, view_list = view_list_plot, range_start = -200, range_end = 200, bin_width = 1):

    arr_df_list = []
    
    for view_idx, view in enumerate(view_list):
        subset = df[df.max_view == view]
        # This should actually be on the study level, not videos
        subset = subset[['study', 'days_post_cfr']].drop_duplicates().reset_index(drop = True)

        range_extent = range_end - range_start

        days_hist, edges = np.histogram(subset.days_post_cfr,
                                        bins = int(range_extent/bin_width),
                                        range = [range_start, range_end])

        edges += 0.5
        
        arr_df = pd.DataFrame({'studies': days_hist,
                               'freq': days_hist/np.sum(days_hist),
                               'left': edges[:-1], 'right': edges[1:],
                               'center': edges[:-1]+0.5})

        arr_df['f_freq'] = ['%0.5f' % p for p in arr_df.freq]

        arr_df['view'] = view

        arr_df['color'] = Category10[10][view_idx]

        arr_df_list.append(arr_df)

    hist_df = pd.concat(arr_df_list, ignore_index=True, axis = 0)
    hist_df = hist_df.sort_values(['view', 'left'])

    return hist_df

def make_plot(df):
    p = figure(title = 'Time for echocardiography studies after CFR measurement by views',
               x_axis_label = 'Time post-cfr [days]',
               y_axis_label = 'Echocardiography studies')
    
    hist_dict = {}
    for v, view in enumerate(df.view.unique()):
    
        df_view = df[df.view == view]
        datasource = ColumnDataSource(df_view) 
        label = view.split('_')[-1]
        hist_dict[view] = p.quad(source = datasource, bottom = 0, top = 'studies', 
                                 left = 'left', right = 'right',
                                 color = 'color', fill_alpha = 0.7, 
                                 hover_fill_color = 'color', hover_fill_alpha = 1.0,
                                 legend_label = label, line_color = 'black', 
                                 name = view, muted_alpha = 0.2)
        
        hist_dict[view].visible = True if view == 'a4c' else False
    
    hover = HoverTool(tooltips = [('view', '@view'),
                                  ('days post-cfr', '@center'),
                                  ('echo studies', '@studies'),
                                  ('proportion', '@f_freq')],
                      mode = 'vline',
                      names = list(df.view.unique()))
    
    p.add_tools(hover)
    
    p.legend.location = 'top_right'
    p.legend.title = 'View: click to hide'
    p.legend.click_policy = 'hide'
    p = style(p)
    
    return p

In [17]:
view_list_plot = ['a4c']
hist_df = make_dataset(df = files_cfr, range_start = -30, range_end = 30, bin_width = 1, view_list = view_list_plot)
hist_plot = make_plot(hist_df)
reset_output()
output_notebook()
show(hist_plot)

Loading BokehJS ...

In [18]:
hist_df.head()

,studies,freq,left,right,center,f_freq,view,color
0,6,0.006522,-29.5,-28.5,-29.0,0.00652,a4c,#1f77b4
1,7,0.007609,-28.5,-27.5,-28.0,0.00761,a4c,#1f77b4
2,8,0.008696,-27.5,-26.5,-27.0,0.00870,a4c,#1f77b4
3,4,0.004348,-26.5,-25.5,-26.0,0.00435,a4c,#1f77b4
4,4,0.004348,-25.5,-24.5,-25.0,0.00435,a4c,#1f77b4


In [19]:
reset_output()
output_file(os.path.join(cfr_data_root, 'cfr_echo_time_'+cfr_meta_date+'.html'), title = 'cfr_echo_time_delay')
save(hist_plot)

'/mnt/obi0/andreas/data/cfr/cfr_echo_time_200219.html'